In [1]:
import sys
print(sys.executable)

/home/ubuntu/ExamenBloc2/.venv/bin/python3


# Exploration, nettoyage des données

In [2]:
"""
Configuration des chemins pour le notebook Jupyter
"""
from pathlib import Path  # Pour manipuler les chemins de fichiers de façon indépendante du système d'exploitation
import json                # Pour lire/écrire des fichiers JSON
import math                # Fournit des fonctions mathématiques (ex: arrondis, contrôle de valeurs)
import pandas as pd        # Librairie pour l'analyse et la manipulation de données tabulaires
import numpy as np         # Gestion avancée des tableaux et des valeurs manquantes (np.nan)
import matplotlib.pyplot as plt  # Visualisation de données

# ------------------------------------------------------------------------
# Définition des chemins de projet
# ------------------------------------------------------------------------

# PROJECT_DIR : dossier racine du projet (notebook considéré comme dans un sous-dossier)
PROJECT_DIR = Path.cwd().parent  # chemin du notebook en cours d'exécution

# DATA_DIR : dossier principal pour toutes les données
DATA_DIR = PROJECT_DIR / "data"

# RAW_DIR : dossier pour les données brutes
RAW_DIR = DATA_DIR / "raw"

# PROCESSED_DIR : dossier pour les données nettoyées
PROCESSED_DIR = DATA_DIR / "processed"

# Dataset brut (JSON Lines : 1 JSON par ligne)
RAW_JSONL_PATH = RAW_DIR / "orders_events.jsonl"

# ------------------------------------------------------------------------
# Vérification et création des dossiers si nécessaire
# ------------------------------------------------------------------------
for folder in [DATA_DIR, RAW_DIR, PROCESSED_DIR]:
    folder.mkdir(parents=True, exist_ok=True)

# ------------------------------------------------------------------------
# Affichage des chemins pour vérification
# ------------------------------------------------------------------------
print("Chemins configurés :")
print(f"Data folder      : {DATA_DIR}")
print(f"Raw data folder  : {RAW_DIR}")
print(f"Raw data path    : {RAW_JSONL_PATH}")
print(f"Processed folder : {PROCESSED_DIR}")




Chemins configurés :
Data folder      : /home/ubuntu/ExamenBloc2/data
Raw data folder  : /home/ubuntu/ExamenBloc2/data/raw
Raw data path    : /home/ubuntu/ExamenBloc2/data/raw/orders_events.jsonl
Processed folder : /home/ubuntu/ExamenBloc2/data/processed


In [3]:
with RAW_JSONL_PATH.open('r', encoding='utf-8') as f: # On ouvre le fichier brut
    lines = f.read().splitlines() # On lit toutes les lignes (une ligne = un JSON)
print (lines[0]) # Affiche la première ligne pour vérification
print (len(lines)) # Affiche le nombre total de lignes pour vérification

{"event_id": "4421d292-0f86-4453-bab9-995256885cca", "event_time": "2026-01-08T17:59:18.973023Z", "order_id": "ORD-00563", "customer": {"customer_id": "CUST-0500", "country": "DE"}, "order": {"device": "tablet", "channel": "email", "main_category": "electronics", "n_items": 4, "basket_value": 193.86, "shipping_fee": 0.0, "discount": 0.0, "order_total": 193.86, "is_returned": 0}}
4050


In [4]:
#Lecture des données JSON Lines dans une liste de dictionnaires
data = [json.loads(line) for line in lines]
print(f"Nombre total d'enregistrements chargés : {len(data)}")  # Affiche le nombre total d'enregistrements chargés

Nombre total d'enregistrements chargés : 4050


# Conversion de la liste de dictionnaires en DataFrame

In [5]:
#Conversion de la liste de dictionnaires en DataFrame pandas pour une manipulation plus facile
df = pd.DataFrame(data)
print(f"Dimensions du DataFrame : {df.shape}")  # Affiche les dimensions du DataFrame (lignes, colonnes)
print("Aperçu des premières lignes du DataFrame :")
print(df.head())  # Affiche les premières lignes du DataFrame pour vérification


Dimensions du DataFrame : (4050, 5)
Aperçu des premières lignes du DataFrame :
                               event_id                   event_time  \
0  4421d292-0f86-4453-bab9-995256885cca  2026-01-08T17:59:18.973023Z   
1  798cbc4e-706e-460a-aa59-46e3549fc957  2026-01-21T08:14:19.037883Z   
2  6cf51636-aa0a-4ea6-be74-2bf2b170859b  2026-01-08T07:08:19.025822Z   
3  373b53a7-a484-459d-a157-5426cd99b979  2026-01-10T08:46:19.012009Z   
4  002c11e5-c0b6-4004-ad00-1b981537a126  2026-01-15T16:58:19.023840Z   

    order_id                                       customer  \
0  ORD-00563  {'customer_id': 'CUST-0500', 'country': 'DE'}   
1  ORD-03012  {'customer_id': 'CUST-0458', 'country': 'ES'}   
2  ORD-04779  {'customer_id': 'CUST-0239', 'country': 'DE'}   
3  ORD-01318  {'customer_id': 'CUST-0180', 'country': 'IT'}   
4  ORD-03652  {'customer_id': 'CUST-0171', 'country': 'ES'}   

                                               order  
0  {'device': 'tablet', 'channel': 'email', 'main...  

In [6]:
# Affichage des types de données pour chaque colonne
print("Types de données des colonnes :")
print(df.dtypes)
print(df.columns)

Types de données des colonnes :
event_id      object
event_time    object
order_id      object
customer      object
order         object
dtype: object
Index(['event_id', 'event_time', 'order_id', 'customer', 'order'], dtype='object')


# Aplatissement des colonnes imbriquées (si nécessaire)

In [7]:
def flatten_json_columns(df):
    """
    Aplatit automatiquement toutes les colonnes contenant des dictionnaires.
    Args:
        df (pd.DataFrame): DataFrame avec éventuellement des colonnes de type dict
    Returns:
        pd.DataFrame: DataFrame aplati
    """
    import pandas as pd 
    df_flat = df.copy()
    for col in df_flat.columns:
        # Vérifie si la colonne contient des dictionnaires (au moins une ligne)
        if df_flat[col].apply(lambda x: isinstance(x, dict)).any():
            # Aplatit la colonne
            normalized = pd.json_normalize(df_flat[col])
            # Renomme les colonnes avec un préfixe pour éviter les collisions
            normalized.columns = [f"{col}_{subcol}" for subcol in normalized.columns]
            # Supprime la colonne originale et joint les nouvelles colonnes aplaties
            df_flat = df_flat.drop(columns=[col]).join(normalized)
    return df_flat


In [8]:
# Applique l'aplatissement des colonnes JSON
df_flat = flatten_json_columns(df)
print(f"Dimensions du DataFrame aplati : {df_flat.shape}")  # Affiche les dimensions du DataFrame aplati
print("Aperçu des premières lignes du DataFrame aplati :")
print(df_flat.head())  # Affiche les premières lignes du DataFrame aplati pour vérification
print("Types de données des colonnes :")
print(df_flat.dtypes)

Dimensions du DataFrame aplati : (4050, 14)
Aperçu des premières lignes du DataFrame aplati :
                               event_id                   event_time  \
0  4421d292-0f86-4453-bab9-995256885cca  2026-01-08T17:59:18.973023Z   
1  798cbc4e-706e-460a-aa59-46e3549fc957  2026-01-21T08:14:19.037883Z   
2  6cf51636-aa0a-4ea6-be74-2bf2b170859b  2026-01-08T07:08:19.025822Z   
3  373b53a7-a484-459d-a157-5426cd99b979  2026-01-10T08:46:19.012009Z   
4  002c11e5-c0b6-4004-ad00-1b981537a126  2026-01-15T16:58:19.023840Z   

    order_id customer_customer_id customer_country order_device order_channel  \
0  ORD-00563            CUST-0500               DE       tablet         email   
1  ORD-03012            CUST-0458               ES       tablet         email   
2  ORD-04779            CUST-0239               DE       mobile         email   
3  ORD-01318            CUST-0180               IT       tablet           ads   
4  ORD-03652            CUST-0171               ES       mobile     

# Rapport rapide NA + valeurs uniques (Code)

In [9]:
# isna() crée un tableau booléen : True si la valeur est manquante (NaN), False sinon
# sum() sur des booléens compte les True (True=1, False=0)
na_count = df_flat.isna().sum()

# mean() sur booléens donne le pourcentage (car True=1, False=0)
# *100 pour convertir en pourcentage
na_pct = df_flat.isna().mean() * 100

# On construit un petit DataFrame de synthèse avec le nombre et le % de NA
na_report = pd.DataFrame({
    "na_count": na_count,           # nombre de NA par colonne
    "na_pct": na_pct.round(2)       # % de NA par colonne (arrondi à 2 décimales)
})

# On trie pour voir les colonnes les plus problématiques en premier
na_report = na_report.sort_values("na_count", ascending=False)

# On affiche le rapport
print(na_report)

# On liste quelques colonnes catégorielles probables (texte)
cat_cols = ["customer_country", "order_device", "order_channel", "order_main_category"]

# Pour chaque colonne catégorielle existante, on affiche :
# - le nombre de valeurs uniques
# - les 20 modalités les plus fréquentes
for col in cat_cols:
    if col in df_flat.columns:                          # on vérifie que la colonne existe
        print(f"\n--- Colonne: {col} ---")
        print("Nombre de valeurs uniques :", df_flat[col].nunique(dropna=False))  # dropna=False compte aussi NaN comme une valeur
        display(df_flat[col].value_counts(dropna=False).head(20))                # top 20 modalités


                      na_count  na_pct
order_channel              124    3.06
order_discount              71    1.75
order_id                     0    0.00
customer_customer_id         0    0.00
event_id                     0    0.00
event_time                   0    0.00
order_device                 0    0.00
customer_country             0    0.00
order_n_items                0    0.00
order_main_category          0    0.00
order_basket_value           0    0.00
order_shipping_fee           0    0.00
order_order_total            0    0.00
order_is_returned            0    0.00

--- Colonne: customer_country ---
Nombre de valeurs uniques : 5


customer_country
IT    826
BE    816
FR    810
ES    803
DE    795
Name: count, dtype: int64


--- Colonne: order_device ---
Nombre de valeurs uniques : 3


order_device
tablet     1390
mobile     1330
desktop    1330
Name: count, dtype: int64


--- Colonne: order_channel ---
Nombre de valeurs uniques : 6


order_channel
affiliate    836
direct       786
email        781
ads          767
seo          756
None         124
Name: count, dtype: int64


--- Colonne: order_main_category ---
Nombre de valeurs uniques : 5


order_main_category
fashion        853
sports         830
electronics    805
home           785
beauty         777
Name: count, dtype: int64

# Nettoyage léger des colonnes texte (strip / lower / upper)

In [12]:
# On définit les colonnes texte qu'on veut normaliser
# Objectif : éviter des différences artificielles ("Mobile" vs "mobile" vs " mobile ")
text_cols = [
    "event_id",
    "order_id",
    "customer_customer_id",
    "customer_country",
    "order_device",
    "order_channel",
    "order_main_category"
]

# On boucle sur chaque colonne
for col in text_cols:
    if col in df_flat.columns:                            # on s'assure que la colonne est bien présente
        df_flat[col] = df_flat[col].astype("string")      # "string" pandas gère mieux les NA que object classique
        df_flat[col] = df_flat[col].str.strip()           # strip() supprime les espaces au début et à la fin

        # On met en minuscules pour homogénéiser certaines catégories
        # (pas event_id / order_id / customer_id, car ce sont des identifiants)
        if col not in ["event_id", "order_id", "customer_customer_id", "customer_country"]:
            df_flat[col] = df_flat[col].str.lower()       # lower() met le texte en minuscules

# Pour le pays, on préfère des codes ISO en MAJ (DE, ES, IT)
if "customer_country" in df_flat.columns:
    df_flat["customer_country"] = df_flat["customer_country"].str.upper()  # upper() met en majuscules

print(df_flat.dtypes)
print(df_flat.head(10))

event_id                string[python]
event_time                      object
order_id                string[python]
customer_customer_id    string[python]
customer_country        string[python]
order_device            string[python]
order_channel           string[python]
order_main_category     string[python]
order_n_items                    int64
order_basket_value             float64
order_shipping_fee             float64
order_discount                 float64
order_order_total              float64
order_is_returned                int64
dtype: object
                               event_id                   event_time  \
0  4421d292-0f86-4453-bab9-995256885cca  2026-01-08T17:59:18.973023Z   
1  798cbc4e-706e-460a-aa59-46e3549fc957  2026-01-21T08:14:19.037883Z   
2  6cf51636-aa0a-4ea6-be74-2bf2b170859b  2026-01-08T07:08:19.025822Z   
3  373b53a7-a484-459d-a157-5426cd99b979  2026-01-10T08:46:19.012009Z   
4  002c11e5-c0b6-4004-ad00-1b981537a126  2026-01-15T16:58:19.023840Z   
5  6ed2c

# Conversion des types de données du DataFrame

In [17]:
import pandas as pd

df_flat_converted = pd.DataFrame()  # Nouveau DataFrame pour les données converties
# ======================================================
# Conversion des colonnes d'identifiants
# → utilisation du type 'string' pandas (plus robuste que 'object')
# ======================================================
df_flat_converted["event_id"] = df_flat["event_id"].astype("string")
df_flat_converted["order_id"] = df_flat["order_id"].astype("string")
df_flat_converted["customer_customer_id"] = df_flat["customer_customer_id"].astype("string")

# ======================================================
# Conversion de la colonne temporelle
# → transformation de la date ISO 8601 en datetime avec timezone UTC
# pd.to_datetime transforme une colonne texte en type datetime utilisable (tri, extraction d'année, etc.)
# errors="coerce" : si un format n'est pas convertible, pandas met NaT (équivalent datetime de NaN)
# utc=True : force la timezone UTC (cohérent avec le suffixe "Z" dans tes dates)
# ======================================================
df_flat_converted["event_time"] = pd.to_datetime(df_flat["event_time"], errors="coerce", utc=True)

# ======================================================
# Conversion des colonnes catégorielles
# → type 'category' pour réduire la mémoire et faciliter les analyses
# ======================================================
categorical_cols = [
    "customer_country",
    "order_device",
    "order_channel",
    "order_main_category"
]

df_flat_converted[categorical_cols] = df_flat[categorical_cols].astype("category")

# ======================================================
# Conversion des colonnes numériques entières
# → nombre d’articles dans la commande
# ======================================================
df_flat_converted["order_n_items"] = df_flat["order_n_items"].astype("int64")

# ======================================================
# Conversion des colonnes numériques décimales
# → montants financiers (float)
# ======================================================
numeric_cols = [
    "order_basket_value",
    "order_shipping_fee",
    "order_discount",
    "order_order_total"
]

df_flat_converted[numeric_cols] = df_flat[numeric_cols].astype("float64")

# ======================================================
# Conversion de l’indicateur de retour
# → transformation de 0 / 1 en booléen (False / True)
# ======================================================
df_flat_converted["order_is_returned"] = df_flat["order_is_returned"].astype("bool")

# ======================================================
# Vérification finale des types de données
# ======================================================
print(df_flat_converted.dtypes)
print(df_flat_converted.head())


event_id                     string[python]
order_id                     string[python]
customer_customer_id         string[python]
event_time              datetime64[ns, UTC]
customer_country                   category
order_device                       category
order_channel                      category
order_main_category                category
order_n_items                         int64
order_basket_value                  float64
order_shipping_fee                  float64
order_discount                      float64
order_order_total                   float64
order_is_returned                      bool
dtype: object
                               event_id   order_id customer_customer_id  \
0  4421d292-0f86-4453-bab9-995256885cca  ORD-00563            CUST-0500   
1  798cbc4e-706e-460a-aa59-46e3549fc957  ORD-03012            CUST-0458   
2  6cf51636-aa0a-4ea6-be74-2bf2b170859b  ORD-04779            CUST-0239   
3  373b53a7-a484-459d-a157-5426cd99b979  ORD-01318            CUST-018

# Gestion des doublons (duplicated + drop_duplicates)

In [18]:
# Doublons exacts (toutes colonnes)
# duplicated() renvoie une Series booléenne :
# True si la ligne est un doublon exact d'une ligne précédente
# sum() compte le nombre de True => le nombre de doublons
nb_dup_exact = df_flat_converted.duplicated().sum()

# On affiche le nombre de doublons exacts
print("Doublons exacts (toutes colonnes) :", nb_dup_exact)

# drop_duplicates() supprime les doublons
# keep="first" garde la première occurrence et supprime les suivantes
df_flat_converted_unique = df_flat_converted.drop_duplicates(keep="first")

# On affiche la nouvelle taille
print("Dimensions après suppression doublons exacts :", df_flat_converted_unique.shape)


Doublons exacts (toutes colonnes) : 50
Dimensions après suppression doublons exacts : (4000, 14)


# Doublons métier sur event_id (garder le plus récent)

In [19]:
# duplicated(subset=["event_id"]) détecte les doublons uniquement sur event_id
# si event_id doit être unique, on veut n'en garder qu'un seul
nb_dup_event = df_flat_converted_unique.duplicated(subset=["event_id"]).sum()
print("Doublons sur event_id :", nb_dup_event)

# Pour garder l'événement le plus récent, on trie d'abord par event_time
# sort_values("event_time") met les plus anciennes en haut, les plus récentes en bas
df_flat_converted_unique = df_flat_converted_unique.sort_values("event_time")

# drop_duplicates(..., keep="last") garde la dernière occurrence (donc la plus récente après tri)
df_flat_converted_unique = df_flat_converted_unique.drop_duplicates(subset=["event_id"], keep="last")

# On affiche la taille finale après ce dédoublonnage
print("Dimensions après dédoublonnage event_id :", df_flat_converted_unique.shape)


Doublons sur event_id : 0
Dimensions après dédoublonnage event_id : (4000, 14)


# Contrôles de cohérence métier

In [21]:
# On crée un dictionnaire pour stocker des "compteurs" d'erreurs/valeurs suspectes
checks = {}

# (1) order_n_items négatif => suspect
checks["order_n_items_negative"] = (df_flat_converted_unique["order_n_items"] < 0).sum()

# (2) Montants négatifs => suspect (à adapter si tu gères des avoirs/retours en négatif)
money_cols = ["order_basket_value", "order_shipping_fee", "order_discount", "order_order_total"]
for col in money_cols:
    checks[f"{col}_negative"] = (df_flat_converted_unique[col] < 0).sum()

# (3) event_time manquant => difficilement exploitable
checks["event_time_missing"] = df_flat_converted_unique["event_time"].isna().sum()

# On convertit le dict en DataFrame pour l'afficher joliment
print(pd.DataFrame.from_dict(checks, orient="index", columns=["count"]))


                             count
order_n_items_negative           0
order_basket_value_negative      0
order_shipping_fee_negative      0
order_discount_negative          0
order_order_total_negative       0
event_time_missing               0


# Gestion des valeurs manquantes (NaN)

In [24]:
# Fill NA catégoriels (mode ou valeur “unknown”)
df_flat_converted_unique_without_na = df_flat_converted_unique.copy()  # Copie pour remplir les NA
# Pour customer_country, si manquant => "UN" (Unknown)
# Creer la nouvelle categorie 
if "customer_country" in df_flat_converted_unique.columns:
    df_flat_converted_unique["customer_country"] = (
        df_flat_converted_unique["customer_country"]
        .cat.add_categories(["UN"])
        .fillna("UN")
    )

if "customer_country" in df_flat_converted_unique.columns:
    df_flat_converted_unique_without_na["customer_country"] = df_flat_converted_unique["customer_country"].fillna("UN")  # fillna remplace NA par "UN"

# Pour device/channel/category, si manquant => la mode (valeur la plus fréquente)
for col in ["order_device", "order_channel", "order_main_category"]:
    if col in df_flat_converted_unique.columns:
        mode_value = df_flat[col].mode()[0]  # mode() renvoie une Series, on prend la première valeur
        df_flat_converted_unique_without_na[col] = df_flat_converted_unique[col].fillna(mode_value)  # fillna remplace NA par la mode


In [26]:
# Fill NA numériques (stratégie simple et robuste)
# (1) shipping_fee et discount :
# Si absent, il est souvent raisonnable de considérer 0 (pas de frais / pas de remise)
for col in ["order_shipping_fee", "order_discount"]:
    if col in df_flat_converted_unique.columns:
        df_flat_converted_unique_without_na[col] = df_flat_converted_unique[col].fillna(0.0)  # remplace NA par 0.0 (float)

# (2) order_n_items :
# On remplace par la médiane (plus robuste que la moyenne si outliers)
if "order_n_items" in df_flat_converted_unique.columns:
    median_items = df_flat_converted_unique["order_n_items"].median()     # calcule la médiane
    df_flat_converted_unique_without_na["order_n_items"] = df_flat_converted_unique["order_n_items"].fillna(median_items)  # remplace NA par médiane
    df_flat_converted_unique_without_na["order_n_items"] = df_flat_converted_unique["order_n_items"].round().astype(int)   # arrondi puis cast en int

# (3) order_basket_value :
# Si absent, on remplace par la médiane (stratégie baseline)
if "order_basket_value" in df_flat_converted_unique.columns:
    median_basket = df_flat_converted_unique["order_basket_value"].median()                  # calcule médiane
    df_flat_converted_unique_without_na["order_basket_value"] = df_flat_converted_unique["order_basket_value"].fillna(median_basket)  # remplace NA

print (df_flat_converted_unique_without_na.dtypes)
print (df_flat_converted_unique_without_na.head())
print (df_flat_converted_unique_without_na.info())

event_id                     string[python]
order_id                     string[python]
customer_customer_id         string[python]
event_time              datetime64[ns, UTC]
customer_country                   category
order_device                       category
order_channel                      category
order_main_category                category
order_n_items                         int64
order_basket_value                  float64
order_shipping_fee                  float64
order_discount                      float64
order_order_total                   float64
order_is_returned                      bool
dtype: object
                                  event_id   order_id customer_customer_id  \
3246  924d78f0-8ead-4a1d-960a-444e1ce458fa  ORD-03149            CUST-0037   
1089  50bf179d-e4ed-4d20-b995-dfaabeb633fe  ORD-03305            CUST-0006   
3323  36df23d9-f1ec-433e-b539-24bce867f5a3  ORD-04082            CUST-0202   
3694  8c60222f-93a7-4cb8-b54f-cf073386ff1b  ORD-00440     

# Cohérence total = basket + shipping - discount

In [27]:
# On calcule un total attendu à partir des composantes
# (basket_value + shipping_fee - discount)
df_flat_converted_unique_without_na["order_total_expected"] = (
    df_flat_converted_unique_without_na["order_basket_value"]          # valeur panier
    + df_flat_converted_unique_without_na["order_shipping_fee"]        # frais de livraison
    - df_flat_converted_unique_without_na["order_discount"]            # remise
)

# On calcule l'écart entre le total présent et le total attendu
df_flat_converted_unique_without_na["order_total_delta"] = (
    df_flat_converted_unique_without_na["order_order_total"] 
    - df_flat_converted_unique_without_na["order_total_expected"]
)
# On définit une tolérance d'arrondi (ex: 1 centime)
tolerance = 0.01

# On crée un flag : True si incohérence (écart > tolérance), False sinon
df_flat_converted_unique_without_na["order_total_inconsistent"] = df_flat_converted_unique_without_na["order_total_delta"].abs() > tolerance

# On compte combien de order_order_total sont manquants avant correction
missing_total_before = df_flat_converted_unique_without_na["order_order_total"].isna().sum()
print("order_order_total manquants (avant fill) :", missing_total_before)

# Si order_order_total est NA, on le remplace par le total attendu calculé
df_flat_converted_unique_without_na["order_order_total"] = df_flat_converted_unique_without_na["order_order_total"].fillna(df_flat_converted_unique_without_na["order_total_expected"])
# On affiche combien de lignes sont incohérentes
print("Nombre de lignes avec total incohérent :", df_flat_converted_unique_without_na["order_total_inconsistent"].sum())

# On montre quelques exemples incohérents pour investigation
print(df_flat_converted_unique_without_na[df_flat_converted_unique_without_na["order_total_inconsistent"]].head(20))

order_order_total manquants (avant fill) : 0
Nombre de lignes avec total incohérent : 28
                                  event_id   order_id customer_customer_id  \
3811  ef3152e6-4c8b-4165-b976-8bc0b129f223  ORD-02570            CUST-0254   
1443  efc90039-8981-4239-a7ec-2d84e62b80a2  ORD-00954            CUST-0456   
1280  53dbcc00-a8e8-4637-beb3-7b4d337a30d9  ORD-04461            CUST-0141   
2563  ee866e1f-37e9-4ef4-b6d1-ada229541929  ORD-01883            CUST-0086   
1360  92a28211-394a-4c6f-b689-6e3bae1c1c1a  ORD-03794            CUST-0384   
3893  b1bd0a4f-844d-4ac1-bdd0-1575d1d0c357  ORD-00743            CUST-0394   
1234  884e2cee-6803-4732-9b59-6f06e6799191  ORD-03696            CUST-0196   
898   b68388d6-b572-4574-8f24-dde47174c47e  ORD-00251            CUST-0310   
3030  2d0e27dc-52ce-408d-94b0-b488ea8575d7  ORD-00354            CUST-0454   
2737  ef16e27f-f81c-47ab-ad19-c6ee07d821ba  ORD-02704            CUST-0543   
1317  deca719b-c768-4cf5-8538-f9fb6974f1b0  ORD-01221

# Apply / Lambda (exemples concrets)

In [28]:

df_final =df_flat_converted_unique_without_na.copy()  # Copie pour transformations finales
# Extraire des features temporelles
# dt.year / dt.month / dt.day / dt.hour :
# fonctionne uniquement si event_time est bien en datetime
df_final["event_year"] = df_final["event_time"].dt.year      # extrait l'année
df_final["event_month"] = df_final["event_time"].dt.month    # extrait le mois
df_final["event_day"] = df_final["event_time"].dt.day        # extrait le jour
df_final["event_hour"] = df_final["event_time"].dt.hour      # extrait l'heure

# On affiche quelques lignes pour vérifier
print(df_final[["event_time", "event_year", "event_month", "event_day", "event_hour"]].head(10))
print(df_final.head(10))

                           event_time  event_year  event_month  event_day  \
3246 2025-12-31 15:53:19.021865+00:00        2025           12         31   
1089 2025-12-31 15:57:19.033003+00:00        2025           12         31   
3323 2025-12-31 15:58:19.027022+00:00        2025           12         31   
3694 2025-12-31 16:06:18.971905+00:00        2025           12         31   
3915 2025-12-31 16:12:19.035292+00:00        2025           12         31   
3190 2025-12-31 16:15:18.996741+00:00        2025           12         31   
1191 2025-12-31 16:27:19.029491+00:00        2025           12         31   
2358 2025-12-31 16:35:19.041179+00:00        2025           12         31   
3486 2025-12-31 16:38:18.977270+00:00        2025           12         31   
621  2025-12-31 16:41:19.002645+00:00        2025           12         31   

      event_hour  
3246          15  
1089          15  
3323          15  
3694          16  
3915          16  
3190          16  
1191          16  


# Apply + lambda sur lignes : prix moyen par article

In [29]:
# On crée une nouvelle colonne "price_per_item"
# = order_basket_value / order_n_items
# On utilise apply + lambda pour appliquer la fonction ligne par ligne
df_final["price_per_item"] = df_final.apply(
    lambda row: row["order_basket_value"] / row["order_n_items"] if row["order_n_items"] > 0 else np.nan,
    axis=1  # axis=1 signifie qu'on applique la fonction sur les lignes
)
# On affiche quelques lignes pour vérifier
print(df_final[["order_basket_value", "order_n_items", "price_per_item"]].head(10))

      order_basket_value  order_n_items  price_per_item
3246              498.90              8       62.362500
1089              491.11              2      245.555000
3323              287.46              8       35.932500
3694               95.67              8       11.958750
3915              364.16              5       72.832000
3190              258.06              3       86.020000
1191              112.74              5       22.548000
2358              249.75              4       62.437500
3486              389.24              7       55.605714
621               121.61              5       24.322000


# Vérif NA finale + types + aperçu

In [30]:
print("Vérification finale des NA :")
print(df_final.isna().sum())    # Affiche le nombre de NA par colonne
print("Types de données finaux :")
print(df_final.dtypes)          # Affiche les types de données finaux
print("Vérifie si event_id est unique (souvent attendu)")
print(df_final["event_id"].nunique() == df_final.shape[0])  # Vérifie si tous les event_id sont uniques
print("Aperçu final des données :")
print(df_final.head())          # Affiche les premières lignes du DataFrame final
print(f"Dimensions finales du DataFrame : {df_final.shape}")  # Affiche les dimensions finales du DataFrame

Vérification finale des NA :
event_id                    0
order_id                    0
customer_customer_id        0
event_time                  0
customer_country            0
order_device                0
order_channel               0
order_main_category         0
order_n_items               0
order_basket_value          0
order_shipping_fee          0
order_discount              0
order_order_total           0
order_is_returned           0
order_total_expected        0
order_total_delta           0
order_total_inconsistent    0
event_year                  0
event_month                 0
event_day                   0
event_hour                  0
price_per_item              0
dtype: int64
Types de données finaux :
event_id                         string[python]
order_id                         string[python]
customer_customer_id             string[python]
event_time                  datetime64[ns, UTC]
customer_country                       category
order_device                    